# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [9]:
import pandas as pd

data = pd.read_csv('labeled.csv')
data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [442]:
data['toxic'].value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from razdel import tokenize
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.1, shuffle=True)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

## Стандартная токенизация

In [42]:
cv_default = CountVectorizer()

X_1 = cv_default.fit_transform(train.comment)
X_test_1 = cv_default.transform(test.comment)

y_1 = train.toxic.values
y_test_1 = test.toxic.values

In [43]:
from sklearn.linear_model import LogisticRegression

clf_1 = LogisticRegression(C=0.1, class_weight='balanced')
clf_1.fit(X_1, y_1)

preds_1 = clf.predict(X_test_1)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Токенизация razdel.tokenize

In [44]:
cv_with_tokenizer = CountVectorizer(tokenizer=lambda text: [_.text for _ in list(tokenize(text))])

X_2 = cv_with_tokenizer.fit_transform(train.comment)
X_test_2 = cv_with_tokenizer.transform(test.comment)

y_2 = train.toxic.values
y_test_2 = test.toxic.values

In [45]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=0.1, class_weight='balanced')
clf.fit(X_2, y_2)

preds_2 = clf.predict(X_test_2)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Сравнение результатов на метриках

In [443]:
from sklearn.metrics import classification_report, f1_score

print("Стандартная токенизация:\n", classification_report(y_test_1, preds_1))
print(f"F1: {f1_score(y_test_1, preds_1)}")
print("\nТокенизация razdel.tokenize:\n", classification_report(y_test_2, preds_2))
print(f"F1: {f1_score(y_test_2, preds_2)}")

Стандартная токенизация:
               precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       960
         1.0       0.73      0.82      0.77       482

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.84      1442

F1: 0.7746341463414634

Токенизация razdel.tokenize:
               precision    recall  f1-score   support

         0.0       0.91      0.85      0.88       960
         1.0       0.73      0.84      0.78       482

    accuracy                           0.84      1442
   macro avg       0.82      0.84      0.83      1442
weighted avg       0.85      0.84      0.85      1442

F1: 0.7833655705996132


Общий F1 выше с токенизацией razdel.tokenize

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [560]:
import numpy as np

docs = [
    "я и ты",
    "ты и я",
    "я, я и только я",
    "только не я",
    "он"
]

tokens = [
    "я", "ты", "и", "только", "не", "он"
]

def return_tf(term, document):
    return(document.count(term))

def return_idf(term, docs):
    df = 0
    for d in docs:
        if term in d:
            df += 1
            continue
    return np.log(len(docs) / df)

def return_tf_x_idf(term, document, docs):
    return return_tf(term, document) * return_idf(term, docs)

table = pd.DataFrame()

for t in tokens:
    tmp = []
    for d in docs:
        tmp.append(return_tf_x_idf(t, d, docs))
    table[t] = tmp

table.index = docs

In [561]:
table

,я,ты,и,только,не,он
я и ты,0.223144,0.916291,0.510826,0.000000,0.000000,0.000000
ты и я,0.223144,0.916291,0.510826,0.000000,0.000000,0.000000
"я, я и только я",0.669431,0.000000,0.510826,0.916291,0.000000,0.000000
только не я,0.223144,0.000000,0.000000,0.916291,1.609438,0.000000
он,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

## CountVectorizer + KNeighborsClassifier

In [659]:
from nltk.corpus import stopwords
from pymystem3 import Mystem

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['весь', 'свой', 'это', 'тебе', 'очень', 'просто', 'ещё', 'почему'])

mystem = Mystem()

In [660]:
from sklearn.neighbors import KNeighborsClassifier

def preprocessor(string):
    lemmatized = mystem.lemmatize(string)
    result = []
    for item in lemmatized:
        if not re.match("\d|\W", item):
            result.append(item)
    return " ".join(result)

cv = CountVectorizer(
    tokenizer=lambda text: [_.text for _ in list(tokenize(text))],
    min_df=30,
    ngram_range=(1,2),    
    preprocessor=preprocessor,
    stop_words=russian_stopwords
)

X = cv.fit_transform(train.comment)
X_test = cv.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [661]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=0, fit_prior=True)

clf.fit(X, y)

preds = clf.predict(X_test)
preds_proba = clf.predict_proba(X_test)

print(classification_report(preds, y_test))

              precision    recall  f1-score   support

         0.0       0.91      0.83      0.87      1056
         1.0       0.62      0.77      0.69       386

    accuracy                           0.81      1442
   macro avg       0.76      0.80      0.78      1442
weighted avg       0.83      0.81      0.82      1442



/opt/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:485: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [662]:
test['preds'] = preds
test['probs_0'] = [el[0] for el in preds_proba]
test['probs_1'] = [el[1] for el in preds_proba]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [663]:
test.sort_values(by='probs_1', ascending=False)[:10]

,index,comment,toxic,preds,probs,probs_0,probs_1
526,3049,открытому пиздабольству детектятся именно русс...,1.0,1.0,"[9.826090456951407e-07, 0.9999990173909548]",6.593971e-17,1.0
964,2251,Несправедливый раздел Русские себе почти всё з...,1.0,1.0,"[4.2482473266546404e-08, 0.9999999575175034]",1.284574e-17,1.0
236,666,"Лол, совковая пидораха полыхает, но аргументов...",1.0,1.0,"[7.921800107823326e-09, 0.9999999920782017]",6.114727e-29,1.0
1000,14253,"Надо просто Тарасов-пидорашек из по выгнать, т...",1.0,1.0,"[6.24795951070416e-06, 0.9999937520404885]",3.591614e-16,1.0
256,886,"Хохлы - отражение русни в кривом зеркале, вобр...",1.0,1.0,"[1.2133374498764735e-08, 0.9999999878666302]",1.365690e-18,1.0
706,449,2:30 - малолетнему дебилу открылся дзен и приш...,1.0,1.0,"[4.3474283675502336e-07, 0.9999995652571648]",2.372544e-17,1.0
925,6096,"Какие же пиндосы дегенераты, пиздец просто.\n",1.0,1.0,"[0.0007222981992673935, 0.9992777018007342]",9.366644e-14,1.0
1437,14053,Чушкаина это и есть пидорашка в ее самом худше...,1.0,1.0,"[0.0015451634077258399, 0.9984548365922702]",1.554988e-13,1.0
540,2163,"ВЕСЬ МИР С НАМИ - прохрюкала нищая пидорашка, ...",1.0,1.0,"[0.0024115181276831, 0.9975884818723155]",2.232096e-13,1.0
318,5558,"Политачеры, вы что ебанутые? Какого хуя я вижу...",1.0,1.0,"[9.393071583707656e-13, 0.9999999999990337]",2.058298e-13,1.0


## TfidfVectorizer + DecisionTreeClassifier

In [655]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(
    preprocessor=preprocessor,
    min_df=50,
    ngram_range=(1,2), 
    sublinear_tf=False,
    stop_words=russian_stopwords
)

X = tf.fit_transform(train.comment)
X_test = tf.transform(test.comment)

y = train.toxic.values
y_test = test.toxic.values

In [656]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(
    criterion='entropy',
    splitter='best'
)

clf.fit(X, y)
preds = clf.predict(X_test)
preds_probs = clf.predict_proba(X_test)
print(classification_report(preds, y_test))

              precision    recall  f1-score   support

         0.0       0.81      0.84      0.82       919
         1.0       0.70      0.64      0.67       523

    accuracy                           0.77      1442
   macro avg       0.75      0.74      0.75      1442
weighted avg       0.77      0.77      0.77      1442



In [657]:
test['preds'] = preds
test['probs_0'] = [el[0] for el in preds_proba]
test['probs_1'] = [el[1] for el in preds_proba]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [658]:
test.sort_values(by='probs_1', ascending=False)[:10]

,index,comment,toxic,preds,probs,probs_0,probs_1
299,2000,орет с дегенератов постит Сартра Самокритично\n,1.0,1.0,"[0.01096420223704809, 0.9890357977629525]",2.728218e-12,1.000000
925,6096,"Какие же пиндосы дегенераты, пиздец просто.\n",1.0,1.0,"[0.0007222981992673935, 0.9992777018007342]",4.967639e-10,1.000000
735,6235,И НЕ ПОЛНЫЙ ДЕГЕНЕРАТ мечтает о пониленде педо...,1.0,1.0,"[0.013448656094922842, 0.9865513439050765]",2.590783e-09,1.000000
526,3049,открытому пиздабольству детектятся именно русс...,1.0,1.0,"[9.826090456951407e-07, 0.9999990173909548]",1.745070e-06,0.999998
706,449,2:30 - малолетнему дебилу открылся дзен и приш...,1.0,1.0,"[4.3474283675502336e-07, 0.9999995652571648]",4.911227e-05,0.999951
1169,589,"Ебать вы тупые дебилы, ой блять\n",1.0,1.0,"[1.3246551106194424e-06, 0.9999986753448907]",1.928192e-03,0.998072
1006,2978,Хохлы одним своим существованием оскорбляют ру...,1.0,1.0,"[8.564699776766148e-06, 0.9999914353002229]",2.082525e-03,0.997917
373,14356,ДА КАКОГО ЕБАНОГО ХУЯ МНЕ ТЕПЕРЬ ЮТУБ РЕКОМЕНД...,1.0,1.0,"[1.303835092140478e-12, 0.9999999999986926]",2.391622e-03,0.997608
36,6497,"null 0 Сука, какие же коммибляди тупые.\n",1.0,1.0,"[0.0008319029025564182, 0.9991680970974438]",2.422966e-03,0.997577
333,2080,"НУ ВСЕ, сука говго щас модераторам пишу твою т...",1.0,1.0,"[2.920826301472967e-07, 0.9999997079173681]",4.151330e-03,0.995849


В оба результата попали только токсичные комментарии. Комментарии между двумя таблицами практически отличны, но везде много ненормативной лексики. Есть общие комментарии – например, "Какие же пиндосы дегенераты, пиздец просто." встречается в обоих таблицах

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

In [639]:
import sklearn

tf = TfidfVectorizer(
    max_df=0.8,
    stop_words=russian_stopwords
)

X = tf.fit_transform(train.comment)

def return_top5_toxic_words(clf):
    clf.fit(X, y)
    feature_importance = pd.DataFrame()
    feature_importance['feature'] = tf.get_feature_names()
    if isinstance(clf, sklearn.linear_model.logistic.LogisticRegression) or isinstance(clf, sklearn.naive_bayes.MultinomialNB):
        feature_importance['importance'] = clf.coef_[0]
    else:    
        feature_importance['importance'] = clf.feature_importances_
    return feature_importance.sort_values(by='importance', ascending=False)[:5]

In [620]:
from sklearn.tree import DecisionTreeClassifier

return_top5_toxic_words(DecisionTreeClassifier())

,feature,importance
60625,хохлы,0.016008
29486,нахуй,0.011975
60593,хохлов,0.011692
5308,блядь,0.008171
5310,блять,0.007217


In [640]:
sklearn.naive_bayes.MultinomialNB

return_top5_toxic_words(MultinomialNB())

,feature,importance
60622,хохлы,-7.851612
60590,хохлов,-7.964018
29485,нахуй,-8.131140
60852,хуй,-8.291794
5308,блядь,-8.319525


In [630]:
from sklearn.linear_model import LogisticRegression

return_top5_toxic_words(LogisticRegression())

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,feature,importance
60625,хохлы,4.876556
60593,хохлов,4.466427
29486,нахуй,3.773715
37243,пиздец,3.275592
5310,блять,3.241132


In [631]:
from sklearn.ensemble import RandomForestClassifier

return_top5_toxic_words(RandomForestClassifier())

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,feature,importance
60625,хохлы,0.009268
60593,хохлов,0.007300
29486,нахуй,0.006744
5308,блядь,0.005668
37243,пиздец,0.004189
